In [2]:
import pypdfium2 as pdfium
from pathlib import Path
from PIL import Image, ImageDraw
import json

In [3]:
def invert_y(rect, y_max):
  return (rect[0], y_max - rect[1], rect[2], y_max - rect[3])

In [4]:
card_packs = {
  'on-the-rocks': {
    'name': 'On The Rocks',
    'cards': ['Test?'],
  },
  'extra-dirty': {
    'name': 'Extra Dirty',
    'cards': [],
    },
  'happy-hour': {
    'name': 'Happy Hour',
    'cards': [],
    },
  'last-call': {
    'name': 'Last Call',
    'cards': [],
    },
}

In [5]:
X_STEP = 197
Y_STEP = 281
X_0 = 20
Y_0 = 610
CARD_WIDTH = 170
CARD_HEIGHT = 185
WIDTH = 596
HEIGHT = 842

for pack in card_packs.keys():
  print(f"Now processing {card_packs[pack]['name']} ('cards-{pack}.pdf')")
  pdf = pdfium.PdfDocument(str(Path('pdfs') / f'cards-{pack}.pdf'))

  for page_idx in range(len(pdf)):
    text_page = pdf.get_textpage(page_idx)

    pil_image = pdf.render_page_topil(page_idx)
    drawer = ImageDraw.Draw(pil_image)

    for y in range(Y_0, 0, -Y_STEP):
      for x in range(X_0, WIDTH, X_STEP):
        rect = (x, y, x + CARD_WIDTH, y + CARD_HEIGHT)
        drawer.rectangle(
          invert_y(rect, HEIGHT),
          outline=(255,0,0),
        )

        card_packs[pack]['cards'].append(text_page.get_text(*rect))
    
    pil_image.show()

Now processing On The Rocks ('cards-on-the-rocks.pdf')
Now processing Extra Dirty ('cards-extra-dirty.pdf')


In [102]:
with open('packs.json', 'w') as out_file:
  json.dump(card_packs, out_file)